We can also visualize our topic models in an interactive way in order to look at each topic
and the theme conveyed by leveraging the pyLDAvis framework

Typically, dimension
reduction techniques like MDS, PDA, and t-SNE are used to visualize the topics in a two-­
dimensional visual.

In [27]:
!pip install pyLDAvis

  Using cached pyLDAvis-3.3.1-py2.py3-none-any.whl
  Using cached funcy-1.16-py2.py3-none-any.whl (32 kB)
  Using cached sklearn-0.0-py2.py3-none-any.whl


In [31]:
!pip install dill

/home/tevo/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)


In [33]:
import pyLDAvis
import warnings
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis

/home/tevo/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
import nltk, os
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF


DATA_PATH = 'nipstxt/'
print(os.listdir(DATA_PATH))

folders = ["nips{0:02}".format(i) for i in range(0,13)]
# Read all texts into a list.
papers = []
for folder in folders:
    file_names = os.listdir(DATA_PATH + folder)
    for file_name in file_names:
        with open(DATA_PATH + folder + '/' + file_name, encoding='utf-8', errors='ignore', mode='r+') as f:
            data = f.read()
        papers.append(data)
len(papers)
stop_words = nltk.corpus.stopwords.words('english')
wtk = nltk.tokenize.RegexpTokenizer(r'\w+')
wnl = nltk.stem.wordnet.WordNetLemmatizer()

def normalize_corpus(papers):
    norm_papers = []
    for paper in papers:
        paper = paper.lower()
        paper_tokens = [token.strip() for token in wtk.tokenize(paper)]
        paper_tokens = [wnl.lemmatize(token) for token in paper_tokens if not token.isnumeric()]
        paper_tokens = [token for token in paper_tokens if len(token) > 1]
        paper_tokens = [token for token in paper_tokens if token not in stop_words]
        paper_tokens = list(filter(None, paper_tokens))
        if paper_tokens:
            norm_papers.append(paper_tokens)
            
    return norm_papers
    
norm_papers = normalize_corpus(papers)
print(len(norm_papers))

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=20, max_df=0.6, ngram_range=(1,2),
                     token_pattern=None, tokenizer=lambda doc: doc,
                     preprocessor=lambda doc: doc)
cv_features = cv.fit_transform(norm_papers)
cv_features.shape

vocabulary = np.array(cv.get_feature_names())
print('Total Vocabulary Size:', len(vocabulary))

TOTAL_TOPICS = 20
nmf_model = NMF(n_components=TOTAL_TOPICS, solver='cd', max_iter=500, random_state=42, alpha=.1, l1_ratio=.85)
document_topics = nmf_model.fit_transform(cv_features)

top_terms = 20
topic_terms = nmf_model.components_
topic_key_term_idxs = np.argsort(-np.absolute(topic_terms), axis=1)[:, :top_terms]
topic_keyterms = vocabulary[topic_key_term_idxs]
topics = [', '.join(topic) for topic in topic_keyterms]
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame(topics,
                         columns = ['Terms per Topic'],
                         index=['Topic'+str(t) for t in range(1, TOTAL_TOPICS+1)])
topics_df

/home/tevo/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['orig', 'nips06', 'nips08', 'nips00', 'nips04', 'idx', 'nips02', 'README_yann', 'nips10', 'nips11', 'nips01', 'nips03', 'nips09', 'nips12', 'nips07', 'MATLAB_NOTES', 'nips05', 'RAW_DATA_NOTES']
1740
Total Vocabulary Size: 14408


/home/tevo/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
/home/tevo/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 500 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
<ipython-input-29-fc6dad7242f7>:61: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Terms per Topic
Topic1,"bound, generalization, size, let, optimal, solution, equation, theorem, approximation, gradient, class, xi, loss, rate, matrix, convergence, theory, dimension, sample, minimum"
Topic2,"neuron, synaptic, connection, potential, dynamic, activity, synapsis, excitatory, layer, simulation, synapse, inhibitory, delay, biological, equation, state, et, et al, fig, activation"
Topic3,"state, action, policy, step, optimal, reinforcement, transition, reinforcement learning, probability, reward, dynamic, value function, markov, machine, task, agent, finite, iteration, sequence, decision"
Topic4,"image, face, pixel, recognition, local, distance, scale, digit, texture, filter, scene, vision, facial, pca, edge, transformation, representation, visual, surface, database"
Topic5,"hidden, layer, net, hidden unit, task, hidden layer, architecture, back, trained, propagation, connection, back propagation, activation, representation, generalization, output unit, neural net, training set, test, learn"
Topic6,"cell, firing, head, direction, response, rat, layer, cortex, activity, spatial, synaptic, inhibitory, synapsis, simulation, cue, property, complex, active, lot, cortical"
Topic7,"word, recognition, speech, context, hmm, speaker, speech recognition, character, phoneme, probability, frame, sequence, rate, level, test, acoustic, experiment, letter, segmentation, state"
Topic8,"signal, noise, source, filter, component, frequency, channel, speech, matrix, independent, separation, sound, ica, phase, eeg, blind, auditory, dynamic, delay, fig"
Topic9,"control, controller, trajectory, motor, dynamic, movement, task, forward, feedback, arm, inverse, position, robot, architecture, hand, force, adaptive, change, command, plant"
Topic10,"circuit, chip, current, analog, voltage, vlsi, gate, threshold, transistor, pulse, design, implementation, synapse, bit, digital, device, analog vlsi, pp, cmos, element"


In [34]:
warnings.filterwarnings('ignore')
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(nmf_model, cv_features, cv, mds='mmds')

/home/tevo/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=             x        y  topics  cluster    Freq
topic                                           
15    -0.30247 -0.06971  1       1       8.95360
0     -0.20805 0.00608   2       1       8.71071
4     -0.13256 -0.04351  3       1       6.95668
2     -0.25883 -0.21791  4       1       6.81122
13    -0.05947 0.20313   5       1       5.56853
7     0.07302  -0.10790  6       1       5.51864
19    -0.01776 0.03172   7       1       5.50077
8     -0.09470 -0.19566  8       1       5.27666
1     -0.00563 -0.35221  9       1       5.18443
16    0.13444  -0.27597  10      1       5.01004
3     0.17638  0.04143   11      1       4.59487
6     0.02318  0.29404   12      1       4.29655
12    -0.31671 0.10288   13      1       4.22112
9     0.20785  0.14395   14      1       3.93037
11    -0.18360 0.14268   15      1       3.89301
14    -0.14348 0.35203   16      1       3.64761
5     0.21096  -0.25354  17      1       3.45108
10    0.30391  -0.17025  18      1       3.30134
17    0.33939  0.06033   19      1       2.75417
18    0.25413  0.30840   20      1       2.41859, topic_info=           Term         Freq        Total Category  logprob  loglift
8195   neuron   10,742.00000 10,742.00000  Default 30.00000 30.00000
4180   feature  9,040.00000  9,040.00000   Default 29.00000 29.00000
5595   image    7,949.00000  7,949.00000   Default 28.00000 28.00000
11919  state    11,536.00000 11,536.00000  Default 27.00000 27.00000
1607   cell     6,981.00000  6,981.00000   Default 26.00000 26.00000
...     ...             ...          ...       ...      ...      ...
1607   cell     504.77401    6,981.51418   Topic20 -4.78050 1.09510 
11030  search   269.14464    1,926.46731   Topic20 -5.40930 1.75380 
3460   effect   261.01871    2,629.98565   Topic20 -5.44000 1.41180 
12265  surface  196.55021    1,048.58406   Topic20 -5.72370 2.04770 
9518   positive 207.19735    1,565.02657   Topic20 -5.67090 1.70000 

[1335 rows x 6 columns], token_table=       Topic    Freq   Term
term                       
26     1     0.04886  2d   
26     2     0.02967  2d   
26     6     0.07155  2d   
26     10    0.01047  2d   
26     11    0.01571  2d   
...    ..        ...  ..   
14341  2     0.93075  yi xi
14373  2     0.10580  z_   
14373  6     0.09069  z_   
14373  7     0.78598  z_   
14373  14    0.01511  z_   

[6532 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 1, 5, 3, 14, 8, 20, 9, 2, 17, 4, 7, 13, 10, 12, 15, 6, 11, 18, 19])

In [35]:
import dill

with open('nmf_model.pkl', 'wb') as f:
    dill.dump(nmf_model, f)
with open('cv_features.pkl', 'wb') as f:
    dill.dump(cv_features, f)
with open('cv.pkl', 'wb') as f:
    dill.dump(cv, f)

In [36]:
import pandas as pd
import numpy as np

top_terms = 20
TOTAL_TOPICS = 20
vocabulary = np.array(cv.get_feature_names())
topic_terms = nmf_model.components_
topic_key_term_idxs = np.argsort(-np.absolute(topic_terms), axis=1)[:, :top_terms]
topic_keyterms = vocabulary[topic_key_term_idxs]
topics = [', '.join(topic) for topic in topic_keyterms]
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame(topics,
                         columns = ['Terms per Topic'],
                         index=['Topic'+str(t) for t in range(1, TOTAL_TOPICS+1)])
topics_df

,Terms per Topic
Topic1,"bound, generalization, size, let, optimal, solution, equation, theorem, approximation, gradient, class, xi, loss, rate, matrix, convergence, theory, dimension, sample, minimum"
Topic2,"neuron, synaptic, connection, potential, dynamic, activity, synapsis, excitatory, layer, simulation, synapse, inhibitory, delay, biological, equation, state, et, et al, fig, activation"
Topic3,"state, action, policy, step, optimal, reinforcement, transition, reinforcement learning, probability, reward, dynamic, value function, markov, machine, task, agent, finite, iteration, sequence, decision"
Topic4,"image, face, pixel, recognition, local, distance, scale, digit, texture, filter, scene, vision, facial, pca, edge, transformation, representation, visual, surface, database"
Topic5,"hidden, layer, net, hidden unit, task, hidden layer, architecture, back, trained, propagation, connection, back propagation, activation, representation, generalization, output unit, neural net, training set, test, learn"
Topic6,"cell, firing, head, direction, response, rat, layer, cortex, activity, spatial, synaptic, inhibitory, synapsis, simulation, cue, property, complex, active, lot, cortical"
Topic7,"word, recognition, speech, context, hmm, speaker, speech recognition, character, phoneme, probability, frame, sequence, rate, level, test, acoustic, experiment, letter, segmentation, state"
Topic8,"signal, noise, source, filter, component, frequency, channel, speech, matrix, independent, separation, sound, ica, phase, eeg, blind, auditory, dynamic, delay, fig"
Topic9,"control, controller, trajectory, motor, dynamic, movement, task, forward, feedback, arm, inverse, position, robot, architecture, hand, force, adaptive, change, command, plant"
Topic10,"circuit, chip, current, analog, voltage, vlsi, gate, threshold, transistor, pulse, design, implementation, synapse, bit, digital, device, analog vlsi, pp, cmos, element"


In [37]:
pyLDAvis.sklearn.prepare(nmf_model, cv_features, cv, mds='mmds')

PreparedData(topic_coordinates=             x        y  topics  cluster    Freq
topic                                           
15    -0.30247 -0.06971  1       1       8.95360
0     -0.20805 0.00608   2       1       8.71071
4     -0.13256 -0.04351  3       1       6.95668
2     -0.25883 -0.21791  4       1       6.81122
13    -0.05947 0.20313   5       1       5.56853
7     0.07302  -0.10790  6       1       5.51864
19    -0.01776 0.03172   7       1       5.50077
8     -0.09470 -0.19566  8       1       5.27666
1     -0.00563 -0.35221  9       1       5.18443
16    0.13444  -0.27597  10      1       5.01004
3     0.17638  0.04143   11      1       4.59487
6     0.02318  0.29404   12      1       4.29655
12    -0.31671 0.10288   13      1       4.22112
9     0.20785  0.14395   14      1       3.93037
11    -0.18360 0.14268   15      1       3.89301
14    -0.14348 0.35203   16      1       3.64761
5     0.21096  -0.25354  17      1       3.45108
10    0.30391  -0.17025  18      1       3.30134
17    0.33939  0.06033   19      1       2.75417
18    0.25413  0.30840   20      1       2.41859, topic_info=           Term         Freq        Total Category  logprob  loglift
8195   neuron   10,742.00000 10,742.00000  Default 30.00000 30.00000
4180   feature  9,040.00000  9,040.00000   Default 29.00000 29.00000
5595   image    7,949.00000  7,949.00000   Default 28.00000 28.00000
11919  state    11,536.00000 11,536.00000  Default 27.00000 27.00000
1607   cell     6,981.00000  6,981.00000   Default 26.00000 26.00000
...     ...             ...          ...       ...      ...      ...
1607   cell     504.77401    6,981.51418   Topic20 -4.78050 1.09510 
11030  search   269.14464    1,926.46731   Topic20 -5.40930 1.75380 
3460   effect   261.01871    2,629.98565   Topic20 -5.44000 1.41180 
12265  surface  196.55021    1,048.58406   Topic20 -5.72370 2.04770 
9518   positive 207.19735    1,565.02657   Topic20 -5.67090 1.70000 

[1335 rows x 6 columns], token_table=       Topic    Freq   Term
term                       
26     1     0.04886  2d   
26     2     0.02967  2d   
26     6     0.07155  2d   
26     10    0.01047  2d   
26     11    0.01571  2d   
...    ..        ...  ..   
14341  2     0.93075  yi xi
14373  2     0.10580  z_   
14373  6     0.09069  z_   
14373  7     0.78598  z_   
14373  14    0.01511  z_   

[6532 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 1, 5, 3, 14, 8, 20, 9, 2, 17, 4, 7, 13, 10, 12, 15, 6, 11, 18, 19])

In [38]:
pyLDAvis.sklearn.prepare(nmf_model, cv_features, cv, mds='tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster    Freq
topic                                               
15    -10.87807  10.53873    1       1       8.95360
0     -60.88403  -75.59182   2       1       8.71071
4     -2.33612   -53.28907   3       1       6.95668
2     -61.38532  -12.91463   4       1       6.81122
13    -28.73774  126.29700   5       1       5.56853
7     1.06661    -123.55281  6       1       5.51864
19    45.14072   -7.94787    7       1       5.50077
8     -140.79597 -98.53055   8       1       5.27666
1     61.53168   -83.24056   9       1       5.18443
16    -72.56306  -140.39688  10      1       5.01004
3     15.98449   63.57819    11      1       4.59487
6     -160.85162 68.94514    12      1       4.29655
12    85.96183   51.92294    13      1       4.22112
9     -99.02084  109.93635   14      1       3.93037
11    -176.76413 -8.68168    15      1       3.89301
14    -117.18988 -37.42020   16      1       3.64761
5     108.82992  -22.94620   17      1       3.45108
10    47.58810   121.93481   18      1       3.30134
17    -106.46783 30.17689    19      1       2.75417
18    -50.61380  59.32884    20      1       2.41859, topic_info=           Term         Freq        Total Category  logprob  loglift
8195   neuron   10,742.00000 10,742.00000  Default 30.00000 30.00000
4180   feature  9,040.00000  9,040.00000   Default 29.00000 29.00000
5595   image    7,949.00000  7,949.00000   Default 28.00000 28.00000
11919  state    11,536.00000 11,536.00000  Default 27.00000 27.00000
1607   cell     6,981.00000  6,981.00000   Default 26.00000 26.00000
...     ...             ...          ...       ...      ...      ...
1607   cell     504.77401    6,981.51418   Topic20 -4.78050 1.09510 
11030  search   269.14464    1,926.46731   Topic20 -5.40930 1.75380 
3460   effect   261.01871    2,629.98565   Topic20 -5.44000 1.41180 
12265  surface  196.55021    1,048.58406   Topic20 -5.72370 2.04770 
9518   positive 207.19735    1,565.02657   Topic20 -5.67090 1.70000 

[1335 rows x 6 columns], token_table=       Topic    Freq   Term
term                       
26     1     0.04886  2d   
26     2     0.02967  2d   
26     6     0.07155  2d   
26     10    0.01047  2d   
26     11    0.01571  2d   
...    ..        ...  ..   
14341  2     0.93075  yi xi
14373  2     0.10580  z_   
14373  6     0.09069  z_   
14373  7     0.78598  z_   
14373  14    0.01511  z_   

[6532 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 1, 5, 3, 14, 8, 20, 9, 2, 17, 4, 7, 13, 10, 12, 15, 6, 11, 18, 19])

In [39]:
pyLDAvis.sklearn.prepare(nmf_model, cv_features, cv, mds='pcoa')

PreparedData(topic_coordinates=             x        y  topics  cluster    Freq
topic                                           
15    -0.10885 0.05787   1       1       8.95360
0     -0.08740 0.09901   2       1       8.71071
4     -0.10027 0.03179   3       1       6.95668
2     -0.08596 0.16254   4       1       6.81122
13    -0.09073 -0.08807  5       1       5.56853
7     0.04326  0.06422   6       1       5.51864
19    0.02516  0.04039   7       1       5.50077
8     -0.01514 0.04346   8       1       5.27666
1     0.21110  0.11743   9       1       5.18443
16    0.19494  -0.12636  10      1       5.01004
3     -0.05518 -0.15039  11      1       4.59487
6     -0.13933 -0.05984  12      1       4.29655
12    -0.08843 0.08276   13      1       4.22112
9     0.06099  0.06230   14      1       3.93037
11    -0.06655 0.04576   15      1       3.89301
14    -0.18303 -0.07873  16      1       3.64761
5     0.23299  -0.01928  17      1       3.45108
10    0.19451  0.05853   18      1       3.30134
17    -0.01794 -0.16528  19      1       2.75417
18    0.07586  -0.17812  20      1       2.41859, topic_info=           Term         Freq        Total Category  logprob  loglift
8195   neuron   10,742.00000 10,742.00000  Default 30.00000 30.00000
4180   feature  9,040.00000  9,040.00000   Default 29.00000 29.00000
5595   image    7,949.00000  7,949.00000   Default 28.00000 28.00000
11919  state    11,536.00000 11,536.00000  Default 27.00000 27.00000
1607   cell     6,981.00000  6,981.00000   Default 26.00000 26.00000
...     ...             ...          ...       ...      ...      ...
1607   cell     504.77401    6,981.51418   Topic20 -4.78050 1.09510 
11030  search   269.14464    1,926.46731   Topic20 -5.40930 1.75380 
3460   effect   261.01871    2,629.98565   Topic20 -5.44000 1.41180 
12265  surface  196.55021    1,048.58406   Topic20 -5.72370 2.04770 
9518   positive 207.19735    1,565.02657   Topic20 -5.67090 1.70000 

[1335 rows x 6 columns], token_table=       Topic    Freq   Term
term                       
26     1     0.04886  2d   
26     2     0.02967  2d   
26     6     0.07155  2d   
26     10    0.01047  2d   
26     11    0.01571  2d   
...    ..        ...  ..   
14341  2     0.93075  yi xi
14373  2     0.10580  z_   
14373  6     0.09069  z_   
14373  7     0.78598  z_   
14373  14    0.01511  z_   

[6532 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 1, 5, 3, 14, 8, 20, 9, 2, 17, 4, 7, 13, 10, 12, 15, 6, 11, 18, 19])